In [1]:
# import Libraries

import openai
import langchain
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
import os

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/python/3.12.1/lib/python3.12/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [4]:
doc=read_doc('documents/')
len(doc)

1

In [5]:
doc

[Document(metadata={'producer': 'iText® Core 9.1.0 (AGPL version), pdfHTML 6.1.0 (AGPL version) ©2000-2025 Apryse Group NV; modified using iText® Core 9.1.0 (AGPL version) ©2000-2025 Apryse Group NV', 'creator': 'PyPDF', 'creationdate': '2025-06-14T16:09:15+05:30', 'moddate': '2025-06-14T16:09:15+05:30', 'source': 'documents/Understanding_the_content.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="I see you'd like more details about the Krutrim AI system and its prompt design. Here's a structured \noverview of how the system is calibrated:\nCore Functionality1. \nLanguage processing optimized for Indian contexts•\nReal-time response generation•\nMulti-language capabilities (including Indian regional languages)•\nPrompt Engineering Approach2. \nPre-defined response templates for common queries•\nDynamic adaptation to user questions•\nContext-aware conversation tracking•\nQuality Assurance Mechanisms3. \nAutomated fact-checking protocols•\nBias detection algorithms•\

In [6]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [7]:
documents=chunk_data(docs=doc)
documents

[Document(metadata={'producer': 'iText® Core 9.1.0 (AGPL version), pdfHTML 6.1.0 (AGPL version) ©2000-2025 Apryse Group NV; modified using iText® Core 9.1.0 (AGPL version) ©2000-2025 Apryse Group NV', 'creator': 'PyPDF', 'creationdate': '2025-06-14T16:09:15+05:30', 'moddate': '2025-06-14T16:09:15+05:30', 'source': 'documents/Understanding_the_content.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="I see you'd like more details about the Krutrim AI system and its prompt design. Here's a structured \noverview of how the system is calibrated:\nCore Functionality1. \nLanguage processing optimized for Indian contexts•\nReal-time response generation•\nMulti-language capabilities (including Indian regional languages)•\nPrompt Engineering Approach2. \nPre-defined response templates for common queries•\nDynamic adaptation to user questions•\nContext-aware conversation tracking•\nQuality Assurance Mechanisms3. \nAutomated fact-checking protocols•\nBias detection algorithms•\

In [8]:
len(documents)

2

In [9]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7e546ab01550>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e544ec26c30>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [10]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Check if API key is loaded (without printing the actual key for security)
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API key loaded successfully")
else:
    print("Warning: OpenAI API key not found in environment variables")

OpenAI API key loaded successfully


In [11]:
vectors=embeddings.embed_query("How are you?")
vectors

[-0.01678639091551304,
 -0.012108419090509415,
 0.006708384957164526,
 -0.02596953511238098,
 -0.016144558787345886,
 0.017601024359464645,
 -0.011133327148854733,
 -0.009917547926306725,
 -0.018131772056221962,
 -0.010423608124256134,
 0.02787034772336483,
 0.0016385245835408568,
 -0.007300845813006163,
 -0.011682587675750256,
 0.007239131256937981,
 -0.015391639433801174,
 0.028364064171910286,
 -0.011812188662588596,
 0.014033917337656021,
 -0.020588016137480736,
 0.0024192151613533497,
 0.006350439973175526,
 0.0010391208343207836,
 -0.008195708505809307,
 -0.015910042449831963,
 -0.007782220374792814,
 0.025093186646699905,
 -0.012423164211213589,
 0.022340713068842888,
 -0.025179587304592133,
 0.005628378130495548,
 0.007689648307859898,
 -0.013194597326219082,
 0.004073168616741896,
 0.008825198747217655,
 -0.022340713068842888,
 0.004057739861309528,
 -0.010479152202606201,
 0.020291784778237343,
 -0.006356611382216215,
 0.027055714279413223,
 0.0012558936141431332,
 -0.0052056

In [12]:

len(vectors)

1536

In [13]:
## Vector Search DB In Pinecone
# Initialize Pinecone with new SDK
pc = PineconeClient(api_key=os.environ.get('PINECONE_API_KEY'))
index_name = "langchanivector"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embeddings dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [14]:
index = PineconeVectorStore.from_documents(
    documents=doc,
    embedding=embeddings,
    index_name=index_name,
    pinecone_api_key=os.environ.get('PINECONE_API_KEY')
)


In [15]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [17]:
llm=OpenAI(model="gpt-3.5-turbo-instruct",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

/tmp/ipykernel_14038/356811839.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain=load_qa_chain(llm,chain_type="stuff")


In [18]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [19]:
our_query = "tell me about the Quality Assurance Mechanisms"
answer = retrieve_answers(our_query)
print(answer)

[Document(id='23145183-2603-44ce-ba99-e6b28931d3a5', metadata={'page': 0.0, 'source': '/var/folders/1z/7d17ydn11kg4mz8y20bm6f7c7wxm4z/T/tmpq0oo4twm/Understanding_the_content.pdf'}, page_content='Enterprise-grade security•\nRegular system updates•\nPerformance monitoring tools•\nWould you like me to elaborate on any specific aspect of how the system processes requests or generates \nresponses? I can provide more technical details or practical examples depending on your interest.\nID: 98b38aac-889e-42c0-bdd7-b6fa34a6db5a generated on 14th June 2025'), Document(id='c3051730-4141-4177-b8c4-406816ebc80a', metadata={'page': 0.0, 'source': '/var/folders/1z/7d17ydn11kg4mz8y20bm6f7c7wxm4z/T/tmp_e6_rxmh/Understanding_the_content.pdf'}, page_content='Enterprise-grade security•\nRegular system updates•\nPerformance monitoring tools•\nWould you like me to elaborate on any specific aspect of how the system processes requests or generates \nresponses? I can provide more technical details or practical

/tmp/ipykernel_14038/848985530.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chain.run(input_documents=doc_search,question=query)


 I don't know about the Quality Assurance Mechanisms, as it is not mentioned in the given context. However, based on the mentioned pieces of context, it can be assumed that the system has quality assurance mechanisms in place to ensure enterprise-grade security, regular system updates, and performance monitoring. Would you like me to provide more information on these aspects?
